In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib
print("OK ✅", pd.__version__, sklearn.__version__)

OK ✅ 2.3.3 1.6.1


# Laboratorio 1 — AlpesHearth

## 1. Carga de datos

In [4]:
import pandas as pd

TRAIN_PATH = "Datos Lab 1.csv"
TEST_PATH  = "Datos Test Lab 1.csv"       
TARGET_COL = "CVD Risk Score"    

df = pd.read_csv(TRAIN_PATH)
df_test = pd.read_csv(TEST_PATH, sep=";")

print("Train:", df.shape)
print("Test:", df_test.shape)
df.head()
df.columns


Train: (1639, 24)
Test: (194, 22)


Index(['Patient ID', 'Date of Service', 'Sex', 'Age', 'Weight (kg)',
       'Height (m)', 'BMI', 'Abdominal Circumference (cm)',
       'Blood Pressure (mmHg)', 'Total Cholesterol (mg/dL)', 'HDL (mg/dL)',
       'Fasting Blood Sugar (mg/dL)', 'Smoking Status', 'Diabetes Status',
       'Physical Activity Level', 'Family History of CVD', 'Height (cm)',
       'Waist-to-Height Ratio', 'Systolic BP', 'Diastolic BP',
       'Blood Pressure Category', 'Estimated LDL (mg/dL)', 'CVD Risk Score',
       'CVD Risk Level'],
      dtype='object')

## 2. Exploración y perfilamiento

In [5]:
# Información general
df.info()

# Conteo de valores nulos
df.isna().sum().sort_values(ascending=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1639 entries, 0 to 1638
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Patient ID                    1639 non-null   object 
 1   Date of Service               1639 non-null   object 
 2   Sex                           1639 non-null   object 
 3   Age                           1571 non-null   float64
 4   Weight (kg)                   1566 non-null   float64
 5   Height (m)                    1578 non-null   float64
 6   BMI                           1586 non-null   float64
 7   Abdominal Circumference (cm)  1578 non-null   float64
 8   Blood Pressure (mmHg)         1639 non-null   object 
 9   Total Cholesterol (mg/dL)     1571 non-null   float64
 10  HDL (mg/dL)                   1557 non-null   float64
 11  Fasting Blood Sugar (mg/dL)   1585 non-null   float64
 12  Smoking Status                1639 non-null   object 
 13  Dia

Diastolic BP                    85
HDL (mg/dL)                     82
Waist-to-Height Ratio           76
Weight (kg)                     73
Height (cm)                     68
Age                             68
Total Cholesterol (mg/dL)       68
Height (m)                      61
Abdominal Circumference (cm)    61
Systolic BP                     61
Estimated LDL (mg/dL)           57
Fasting Blood Sugar (mg/dL)     54
BMI                             53
CVD Risk Score                  29
Patient ID                       0
Blood Pressure Category          0
Smoking Status                   0
Family History of CVD            0
Physical Activity Level          0
Diabetes Status                  0
Date of Service                  0
Blood Pressure (mmHg)            0
Sex                              0
CVD Risk Level                   0
dtype: int64

## 3. Preparación de datos

## 4. Modelos (2 pipelines)

## 5. Tabla comparativa (RMSE/MAE/R2)

## 6. Interpretación e importancia de variables

## 7. Respuestas “Análisis de resultados”

## 8. Predicciones en test no etiquetado

## 9. Uso de herramientas de IA generativa